<a target="_blank" href="https://colab.research.google.com/github/vectice/GettingStarted/blob/main/24.1/samples/Validation%20library/ValidationLibrary.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Validation library test



## Install the latest Vectice Python client library

In [1]:
from pathlib import Path
import sys

path_root = Path("./src/vectice").parents[1]
sys.path.append(str(path_root))
print(sys.path)

import vectice

print(path_root)

['/Users/bryandaversa/Vectice/sdk-python/perso', '/Users/bryandaversa/.pyenv/versions/3.9.16/lib/python39.zip', '/Users/bryandaversa/.pyenv/versions/3.9.16/lib/python3.9', '/Users/bryandaversa/.pyenv/versions/3.9.16/lib/python3.9/lib-dynload', '', '/Users/bryandaversa/Vectice/sdk-python/.venv/lib/python3.9/site-packages', '/Users/bryandaversa/Vectice/sdk-python/src', '.']
.


## Run your regular ML project

## Import library

In [1]:
# import piml
import pandas as pd

# numpy and pandas for data manipulation
import numpy as np
import pandas as pd

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder


# File system manangement
import os

# Suppress warnings
import warnings

warnings.filterwarnings("ignore")

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## Feature engineering and Train test split

In [2]:
from sklearn.model_selection import train_test_split

# Training data
application_cleaned = pd.read_csv("/Users/bryandaversa/Vectice/GettingStarted/perso/Demo-Bank/application_train.csv")
app_train_feat, app_test_feat = train_test_split(application_cleaned, test_size=0.15, random_state=42)
# Separate the target variable from the testing set
target_variable = "TARGET"
app_test_feat_target = app_test_feat[target_variable]
app_test_feat = app_test_feat.drop(target_variable, axis=1)

# Print the shapes of the resulting dataframes
print("Training data shape: ", app_train_feat.shape)
print("Testing shape: ", app_test_feat.shape)
print("Testing target shape: ", app_test_feat_target.shape)

Training data shape:  (261384, 122)
Testing shape:  (46127, 121)
Testing target shape:  (46127,)


In [4]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train_feat:
    if app_train_feat[col].dtype == "object":
        # If 2 or fewer unique categories
        if len(list(app_train_feat[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train_feat[col])
            # Transform both training and testing data
            app_train_feat[col] = le.transform(app_train_feat[col])
            app_test_feat[col] = le.transform(app_test_feat[col])

            # Keep track of how many columns were label encoded
            le_count += 1

print("%d columns were label encoded." % le_count)


# one-hot encoding of categorical variables
app_train_feat = pd.get_dummies(app_train_feat)
app_test_feat = pd.get_dummies(app_test_feat)
train_labels = app_train_feat["TARGET"]

# Align the training and testing data, keep only columns present in both dataframes
app_train_feat, app_test_feat = app_train_feat.align(app_test_feat, join="inner", axis=1)

# Add the target back in
app_train_feat["TARGET"] = train_labels


# Create an anomalous flag column
app_train_feat["DAYS_EMPLOYED_ANOM"] = app_train_feat["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train_feat["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)


app_test_feat["DAYS_EMPLOYED_ANOM"] = app_test_feat["DAYS_EMPLOYED"] == 365243
app_test_feat["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)
print("Training Features shape: ", app_train_feat.shape)
print("Testing Features shape: ", app_test_feat.shape)
print(
    "There are %d anomalies in the test data out of %d entries"
    % (app_test_feat["DAYS_EMPLOYED_ANOM"].sum(), len(app_test_feat))
)

3 columns were label encoded.
Training Features shape:  (261384, 244)
Testing Features shape:  (46127, 243)
There are 8287 anomalies in the test data out of 46127 entries


In [5]:
from sklearn.impute import SimpleImputer

# Drop the target from the training data
if "TARGET" in app_train_feat:
    train_no_missing = app_train_feat.drop(columns=["TARGET"])

features = list(train_no_missing.columns)

# Median imputation of missing values
imputer = SimpleImputer(strategy="median")
# Fit on the training data
imputer.fit(train_no_missing)

# Transform both training and testing data
train_no_missing = pd.DataFrame(imputer.transform(train_no_missing), columns=features).set_index("SK_ID_CURR")
test_no_missing = pd.DataFrame(imputer.transform(app_test_feat), columns=features).set_index("SK_ID_CURR")

# Build model

In [7]:
from sklearn.ensemble import RandomForestClassifier

# Make the random forest classifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=50, min_samples_leaf=0.01, n_jobs=-1)
features = list(train_no_missing.columns)
# Train on the training data
random_forest.fit(train_no_missing, train_labels)

# Extract feature importances
# feature_importance_values = random_forest.feature_importances_

# Make predictions on the test data
predictions = random_forest.predict_proba(test_no_missing)[:, 1]
prediction_absolte_threshold = (random_forest.predict_proba(test_no_missing)[:,1] >= 0.1).astype(bool) 

# Validation Library

## Connect to Vectice

In [14]:
import vectice
from vectice.models.validation import ValidationModel

connect = vectice.connect(
    api_token="oQjZ7xVOG.vaJwZbLMYD4WR2z8PN6moQjZ7xVOG35yKqpre7xl91nEkXB0Vd", host="https://qa.vectice.com"
)

phase = connect.phase("PHA-6406")
iteration = phase.create_iteration()

Welcome, bdaversa. You`re now successfully connected to Vectice.

To connect to a phase, copy your phase ID from your Vectice app and use connect.phase("PHA-XXXX")
To create your first iteration do iteration = connect.phase("PHA-XXXX").create_or_get_current_iteration()

If you are using a notebook you can call the help by using a Vectice returned object with the builtin notebook "?":
>> connection?

If you are using an IDE you can call the help() method on any object returned by Vectice:
>> help(connection)

For quick access to your default workspace in the Vectice web app, visit:
https://qa.vectice.com/browse/workspace/WSP-309
Phase 'Modeling' successfully retrieved.

For quick access to the Phase in the Vectice web app, visit:
https://qa.vectice.com/browse/phase/PHA-6406
New Iteration number 60 created.

For quick access to the Iteration in the Vectice web app, visit:
https://qa.vectice.com/browse/iteration/ITR-6694


## Tell us which Model and which test you want to run

In [21]:
validation_suite = ValidationModel(asset="MDV-4090",                        #The asset you want to run the test on
                                dataframe = train_no_missing,               #Your dataset
                                predictor = random_forest,                  #Your predictor
                                target = app_test_feat_target.values,       #The taregt
                                prediction = prediction_absolte_threshold,  #The prediction
                                tests = ["binary_full_suite"],              #The test suite you want to run
                                )


iteration.log(validation_suite)


Test: binary_tests successfully run
Model version 'MDV-4090' attachments successfully updated.
Existing model version: 'MDV-4090' of model: 'PHA-6786-knn' already linked to iteration: 60.
Link to iteration: https://qa.vectice.com/browse/iteration/ITR-6694

